In [5]:
import pandas as pd
import gzip as gz
import pickle as pkl

In [6]:
hasTimeInterval = ['worksAt', 'graduatedFrom',
                   'isMarriedTo', 'playsFor', 'isLeaderOf',
                   'hasAcademicAdvisor', 'isAffiliatedTo']
hasTimeStamp = ['actedIn', 'wroteMusicFor','created', 'participatedIn']

In [ ]:
metafacts = pd.read_csv('actedIn.csv', sep='\t') #, header=None)

# Check if metafacts has data
for r in metafacts:
    print(r)    
print(len(metafacts))

In [8]:
data = {} # temporary data array

# information on columns to use less space to save everything

num_predicates = 0
predicate2idx = {}
idx2predicate = {}

num_objs = 0
obj2idx = {}
idx2obj = {}

cols = ['idx', 'id', 'subject', 'predicate', 'object', 'inDateTime', 'after', 'before']

In [ ]:
for file in hasTimeInterval:
    filename = '%s.csv'%(file)
    print("Proc: ", filename)
    metafacts = pd.read_csv(filename, sep='\t', header=None)
    num_mf = len(metafacts)
    print("Rows: ", num_mf)
    rows = []
    for _, mf in metafacts.iterrows():
        idn, subject, predicate, obj, after, date_after, before, date_before = mf
        if after == 'after':
            if not subject in obj2idx:
                idx2obj[num_objs] = subject
                obj2idx[subject] = num_objs
                num_objs += 1
                
            if not predicate in predicate2idx:
                idx2predicate[num_predicates] = predicate
                predicate2idx[predicate] = num_predicates
                num_predicates += 1
                
            if not obj in obj2idx:
                idx2obj[num_objs] = obj
                obj2idx[obj] = num_objs
                num_objs += 1
                
            s = obj2idx[subject]
            p = predicate2idx[predicate]
            o = obj2idx[obj]
            rows.append({'idx':        '%s%s%s-%s%s'%(subject,predicate,obj,date_after,date_before),
                         'id':         idn,
                         'subject':    s,
                         'predicate':  p,
                         'object':     o,
                         'inDateTime': None,
                         'after':      date_after,
                         'before':     date_before})
    data[file] = pd.DataFrame(rows, columns=cols)
    #data[file] = data[file].set_index(['idx'])
    data[file] = data[file].set_index('idx')
    data[file] = data[file].sort_index()

In [ ]:
for file in hasTimeStamp:
    filename = '%s.csv'%(file)
    print("Proc: ", filename)
    metafacts = pd.read_csv(filename, sep='\t', header=None)
    num_mf = len(metafacts)
    print("Rows: ", num_mf)
    rows = []
    #atom_idx = {}
    for _, mf in metafacts.iterrows():
        idn, subject, predicate, obj, in_date_time, date = mf
        if in_date_time == 'inDateTime':
            if not subject in obj2idx:
                idx2obj[num_objs] = subject
                obj2idx[subject] = num_objs
                num_objs += 1
                
            if not predicate in predicate2idx:
                idx2predicate[num_predicates] = predicate
                predicate2idx[predicate] = num_predicates
                num_predicates += 1
                
            if not obj in obj2idx:
                idx2obj[num_objs] = obj
                obj2idx[obj] = num_objs
                num_objs += 1
                
            s = obj2idx[subject]
            p = predicate2idx[predicate]
            o = obj2idx[obj]
            rows.append({'idx':        '%s%s%s%s--'%(subject,predicate,obj,date),
                         'id':         idn,
                         'subject':    s,
                         'predicate':  p,
                         'object':     o,
                         'inDateTime': date,
                         'after':      None,
                         'before':     None})
    data[file] = pd.DataFrame(rows, columns=cols)
    data[file] = data[file].set_index('idx')
    data[file] = data[file].sort_index()

In [11]:
print(num_predicates, num_objs)

11 41646


In [ ]:
# Test if 'data' was successul created
for r in data['actedIn']:
    print(r)
    
total = 0
for file in data:
    print(file, len(data[file]))
    total += len(data[file])
    
print(total)

In [ ]:
print(data['actedIn'][:2])

In [ ]:
print(data['worksAt'][:2])

## Create the meta-facts input file

In [16]:
data_dict = (data, idx2obj, idx2predicate, predicate2idx)
f = gz.open('metafacts.pkl.gz','wb')
pkl.dump(data_dict,f)
f.close()